In [ ]:
!pip install langchain
!pip install jq
!pip install sentence-transformers

In [1]:
from langchain_community.document_loaders import JSONLoader
from tqdm.auto import tqdm
import json
import pickle
from langchain.text_splitter import Language
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma


In [2]:
with open("../data/chronicle/rag_msg.pkl", "rb") as f:
    msg_data = pickle.load(f)

In [3]:
diff_loader = JSONLoader(
    file_path='../data/chronicle/chronicle_rag_db.json',
    jq_schema='.[].diff',
    text_content=False)

diff_data = diff_loader.load()

In [4]:

language_loader = JSONLoader(
    file_path='../data/chronicle/chronicle_rag_db.json',
    jq_schema='.[].lang',
    text_content=False)

language_data = language_loader.load()

In [5]:
# 1-1000 data use java splitter, 1000-2000 use cpp splitter, 2000-3000 use csharp splitter, 3000-4000 use python splitter, 4000-5000 use javascript splitter


# languages = [Language.JAVA, Language.CPP, Language.CSHARP, Language.PYTHON, Language.JS]
languages = [Language.PYTHON, Language.JS]
splitters = [RecursiveCharacterTextSplitter.from_language(language, chunk_size=500, chunk_overlap=50) for language in languages]

# language_dict = {'java': 0, 'c++': 1, 'c#': 2, 'python': 3, 'javascript': 4}
language_dict = {'Python': 0, 'JavaScript': 1}

In [14]:
def similarity_search(documents, db):
    # Initialize an empty dictionary to store aggregate scores for each candidate ID
    aggregate_scores = {}

    # Iterate through each document in the documents list
    for document in documents:
        # Apply similarity search function to the document
        results = db.similarity_search_with_relevance_scores(document.page_content, score_threshold=0.0)
        # Iterate through the results for each document
        for candidate_doc in results:
            id = candidate_doc[0].metadata['seq_num']
            score = candidate_doc[1]
            # Update the aggregate score for the candidate ID
            aggregate_scores[id] = aggregate_scores.get(id, 0) + score

    # Find the candidate ID with the highest aggregate score
    if aggregate_scores:
        max_candidate_id = max(aggregate_scores, key=aggregate_scores.get)
    else:
        max_candidate_id = -1

    return max_candidate_id

In [12]:
diff_split = []
for i, doc in tqdm(enumerate(diff_data), total=len(diff_data), desc="Processing documents"):
    if language_data[i].page_content=='Python':
        # diff_split.append(doc)
        diff_split += splitters[language_dict[language_data[i].page_content]].split_documents([doc])

Processing documents:   0%|          | 0/500000 [00:00<?, ?it/s]

In [6]:
# Define the path to the pre-trained model you want to use
modelPath = "mixedbread-ai/mxbai-embed-large-v1"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda', 'trust_remote_code': True}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    cache_folder = '../models',
    
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [ ]:
test_diff_loader = JSONLoader(
    file_path='../data/chronicle/rag_dev/chronicle_test_data_dev.json',
    jq_schema='.[].diff',
    text_content=False)

test_diff_data = test_diff_loader.load()

with open('../data/chronicle/rag_dev/chronicle_test_data_dev.json', 'r') as f:
    test_data = json.load(f)

# Python part

In [ ]:
# Load the document, split it into chunks, embed each chunk and load it into the vector store.
db = Chroma.from_documents(diff_split, embeddings, persist_directory="./chroma_chronicle_db_mxbai_500chunk_normalized_python")

In [23]:
python_test_data = []
python_indices = []
for i, item in enumerate(test_data):
    if item['lang'] == 'Python':
        python_test_data.append(item)
        python_indices.append(i)

In [24]:
db = Chroma(persist_directory="./chroma_chronicle_db_mxbai_500chunk_normalized_python", embedding_function=embeddings)
similar_diff = []
for index in tqdm(python_indices, total=len(python_indices), desc="Processing documents"):
    documents = splitters[0].split_documents([test_diff_data[index]])
    similar_diff.append(similarity_search(documents, db))

data = []

for sim_diff, test_diff in zip(similar_diff, python_test_data):
    item = {'org_diff': test_diff['diff'],
            'org_msg': test_diff['msg'],
            'sim_msg': msg_data[sim_diff],
            'sim_diff': diff_data[sim_diff-1].page_content}
    data.append(item)
    
# Write the data to a JSON file
with open('../data/chronicle/rag_baseline/rag_baseline_python.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

Processing documents:   0%|          | 0/1000 [00:00<?, ?it/s]

# JavaScript part

In [25]:
js_test_data = []
js_indices = []
for i, item in enumerate(test_data):
    if item['lang'] == 'JavaScript':
        js_test_data.append(item)
        js_indices.append(i)

In [1]:
db = Chroma(persist_directory="./chroma_chronicle_db_mxbai_500chunk_normalized_js", embedding_function=embeddings)
similar_diff = []
for index in tqdm(js_indices, total=len(js_indices), desc="Processing documents"):
    documents = splitters[1].split_documents([test_diff_data[index]])
    similar_diff.append(similarity_search(documents, db))

data = []

for sim_diff, test_diff in zip(similar_diff, js_test_data):
    item = {'org_diff': test_diff['diff'],
            'org_msg': test_diff['msg'],
            'sim_msg': msg_data[sim_diff],
            'sim_diff': diff_data[sim_diff-1].page_content}
    data.append(item)
    
# Write the data to a JSON file
with open('../data/chronicle/rag_baseline/rag_baseline_js.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

NameError: name 'Chroma' is not defined

In [16]:
diff_split = []
for i, doc in tqdm(enumerate(diff_data), total=len(diff_data), desc="Processing documents"):
    if language_data[i].page_content=='JavaScript':
        # diff_split.append(doc)
        diff_split += splitters[language_dict[language_data[i].page_content]].split_documents([doc])
# Load the document, split it into chunks, embed each chunk and load it into the vector store.
db = Chroma.from_documents(diff_split, embeddings, persist_directory="./chroma_chronicle_db_mxbai_500chunk_normalized_js")

Processing documents:   0%|          | 0/500000 [00:00<?, ?it/s]

In [56]:
similar_diff = []
for i, test_data in tqdm(enumerate(test_diff_data[:100]), total=len(test_diff_data[:100]), desc="Processing documents"):
    query = test_data.page_content
    similar_diff.append(diff_data[retriever.get_relevant_documents(query)[0].metadata['seq_num']-1])

data = []

for sim_diff, test_diff in zip(similar_diff, test_diff_data[:100]):
    item = {
        'sim_msg': msg_data[sim_diff.metadata['seq_num']],
        'sim_diff': sim_diff.page_content,
        'org_diff': test_diff.page_content
    }
    data.append(item)

# Write the data to a JSON file
with open('../data/chronicle/data_with_similar_diff_codebert.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

Processing documents:   0%|          | 0/100 [00:00<?, ?it/s]

In [57]:
similar_diff = []
for i, test_data in tqdm(enumerate(test_diff_data[:100]), total=len(test_diff_data[:100]), desc="Processing documents"):
    query = test_data.page_content
    similar_diff.append(diff_data[retriever_2.get_relevant_documents(query)[0].metadata['seq_num']-1])

data = []

for sim_diff, test_diff in zip(similar_diff, test_diff_data[:100]):
    item = {
        'sim_msg': msg_data[sim_diff.metadata['seq_num']],
        'sim_diff': sim_diff.page_content,
        'org_diff': test_diff.page_content
    }
    data.append(item)

# Write the data to a JSON file
with open('../data/chronicle/data_with_similar_diff_mxbai.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

Processing documents:   0%|          | 0/100 [00:00<?, ?it/s]

In [48]:
data = []

for sim_diff, test_diff in zip(similar_diff, test_diff_data):
    item = {
        'sim_msg': msg_data[sim_diff.metadata['seq_num']],
        'sim_diff': sim_diff.page_content,
        'org_diff': test_diff.page_content
    }
    data.append(item)

In [49]:
# Write the data to a JSON file
with open('../data/chronicle/data_with_similar_diff_codebert.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)